In [ ]:
import JupyterFormatter
JupyterFormatter.enable_autoformat();

In [ ]:
import NBInclude: @nbinclude
@nbinclude("Data.ipynb")

In [ ]:
import Random
import SHA
import StatsBase

In [ ]:
const MIN_ITEMS = 5;
const MAX_VALIDATION_DAYS = 7
const MAX_VALIDATION_ITEMS = 5
const VALIDATION_USER_FRAC = 0.01;

In [ ]:
function drop_sparse_users!(dfs)
    user_counts = Dict()
    @showprogress for df in dfs
        for (k, v) in StatsBase.countmap(df.userid)
            @assert k ∉ keys(user_counts)
            user_counts[k] = v
        end
    end
    valid_userids = Set(k for (k, v) in user_counts if v >= MIN_ITEMS)

    @showprogress Threads.@threads for i = 1:length(dfs)
        dfs[i] = subset(dfs[i], dfs[i].userid .∈ (valid_userids,))
    end
end;

In [ ]:
function relabel_userid_map(dfs)
    userids = Int32[]
    @showprogress for df in dfs
        for k in Set(df.userid)
            push!(userids, k)
        end
    end
    @assert length(userids) == length(Set(userids))
    sort!(userids)
    hash = SHA.sha256(reinterpret(UInt8, userids))
    Random.seed!(first(reinterpret(UInt64, hash)))
    Random.shuffle!(userids)
    userid_map = Dict(u => i for (i, u) in Iterators.enumerate(userids))
    CSV.write(
        get_data_path("processed_data/relabel_userid_map.csv"),
        DataFrames.DataFrame([(k, v) for (k, v) in userid_map], [:username, :userid]),
    )
end;

In [ ]:
function create_splits!(dfs)
    userid_map = read_csv(get_data_path("processed_data/relabel_userid_map.csv"))
    users = userid_map.userid .|> x -> parse(Int32, x)
    validation_users = Set(
        StatsBase.sample(
            users,
            round(Int, length(users) * VALIDATION_USER_FRAC),
            replace = false,
        ),
    )

    train_dfs = Vector{eltype(dfs)}(undef, length(dfs))
    val_dfs = Vector{eltype(dfs)}(undef, length(dfs))
    @showprogress Threads.@threads for t = 1:length(dfs)
        df = dfs[t]
        df = subset(df, sortperm(collect(zip(df.userid, df.updated_at, df.update_order))))
        ts_cutoff = 1 - (24 * 60 * 60 * MAX_VALIDATION_DAYS) / (MAX_TS - MIN_TS)
        userid = nothing
        num_val = [0 for _ in MEDIUM_MAP]
        val_mask = BitArray([false for _ = 1:length(df.userid)])
        for i in reverse(1:length(df.userid))
            if userid != df.userid[i]
                userid = df.userid[i]
                num_val .= 0
            end
            if (df.userid[i] in validation_users) &&
               (df.updated_at[i] > ts_cutoff) &&
               (num_val[df.medium[i]+1] < MAX_VALIDATION_ITEMS)
                num_val[df.medium[i]+1] += 1
                val_mask[i] = true
            end
        end
        train_dfs[t] = subset(df, .!val_mask)
        val_dfs[t] = subset(df, val_mask)
        dfs[t] = RatingsDataset([[] for _ in fieldnames(RatingsDataset)]...) # free memory
    end
    train_dfs, val_dfs
end;

In [ ]:
dfs = load_datasets("training_data");

In [ ]:
drop_sparse_users!(dfs);

In [ ]:
relabel_userid_map(dfs);

In [ ]:
relabel_userids!(dfs);

In [ ]:
train_dfs, val_dfs = create_splits!(dfs);

In [ ]:
save_splits(train_dfs, val_dfs, "training");